In [4]:
! pip install faiss-cpu sentence-transformers PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 34.8 MB/s eta 0:00:00


In [14]:
import fitz  # PyMuPDF
import os

def extract_text_from_pdfs(folder_path):
    documents = []
    filenames = []

    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(folder_path, file)
            doc_text = ""

            # Đọc từng trang trong PDF
            pdf_doc = fitz.open(pdf_path)
            for page in pdf_doc:
                doc_text += page.get_text("text") + "\n"

            # Lưu văn bản và tên file
            documents.append(doc_text)
            filenames.append(file)

    return documents, filenames

# Thư mục chứa các file PDF
pdf_folder = "/content/drive/MyDrive/IELTS"
documents, filenames = extract_text_from_pdfs(pdf_folder)

print(f"Đã trích xuất {len(documents)} tài liệu từ PDF.")

Đã trích xuất 4 tài liệu từ PDF.


In [15]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load model để tạo embedding
model = SentenceTransformer("all-MiniLM-L6-v2")

# Chuyển văn bản thành embedding
doc_embeddings = model.encode(documents, convert_to_numpy=True)

# Khởi tạo FAISS Index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Thêm dữ liệu vào FAISS
index.add(doc_embeddings)

# Lưu FAISS Index ra file
faiss.write_index(index, "document_index.faiss")

# Lưu tên file để sau này truy xuất
with open("filenames.txt", "w") as f:
    for filename in filenames:
        f.write(filename + "\n")

print("Đã lưu FAISS Index thành công!")

Đã lưu FAISS Index thành công!


In [19]:
# Load FAISS Index và danh sách file
index = faiss.read_index("document_index.faiss")
with open("filenames.txt", "r") as f:
    filenames = f.read().splitlines()

def search_pdf(query, top_k=3):
    query_embedding = model.encode([query], convert_to_numpy=True)

    # Tìm các tài liệu gần nhất
    _, closest_doc_indices = index.search(query_embedding, top_k)

    # Lấy nội dung tương ứng
    results = [(filenames[idx], documents[idx]) for idx in closest_doc_indices[0]]
    return results

# Ví dụ câu hỏi
query = "source file layout of Verilog Programming Conventions"
results = search_pdf(query)

print("\n🔍 Kết quả tìm kiếm:")
for filename, content in results:
    print(f"📄 File: {filename}")
    print(f"📜 Nội dung: {content[:50000]}...\n")  # Hiển thị 500 ký tự đầu


🔍 Kết quả tìm kiếm:
📄 File: code.pdf
📜 Nội dung: Arrive Technologies Inc. 
Guidelines 
 
 
This controlled document is the proprietary of Arrive Technologies Inc..  
Any duplication, reproduction, or transmission to unauthorized parties is prohibited.  
Copyright © 2006 
 
ATVN  
Verilog Coding Standards  
 
 
 
 
Abstract: This document describes the Verilog Programming Style to be used in the logic 
development. A Set of conventions is described covering various constructs of the language. These 
conventions are intended to provide a high level of uniformity in the code produced by different 
programmers thus achieving readability and maintainability. 
 
Revision: 
3.1 
Last Updated: 
03-Aug-06 
Author: 
− 
AT ASIC Design Group 
 
 
 

 
Verilog Coding Standards  
ATVN  
 
 
 
Copyright © 2006. Arrive Technologies Inc. 
 
Page i 
Internal Doc. Subject to Change 
 
Document History 
Revision 
Date 
Description 
1.0 
May 2001 
Initial Version 
2.0 
Jan 2003 
Changed for new RED-BLUE c